In [81]:
import openpyxl
import xlrd
import regex as re
from openpyxl import workbook

In [82]:
wb_matches = openpyxl.load_workbook('all_matches.xlsx')
wb_bowler = openpyxl.load_workbook('bowler_data.xlsx')

In [104]:
matches=wb_matches.active
bowler=wb_bowler.active
wb_stadium = openpyxl.load_workbook('stadium_data.xlsx')
stadium=wb_stadium.active

In [105]:
bowlers,stadiums=dict(),dict()
    
for rowx in range(2,bowler.max_row+1):
    player_name1 = str(bowler.cell(row=rowx,column=1).value) 
    player_name = player_name1[0:1] +" "+ player_name1[len(player_name1)-5 : len(player_name1)+1]
    
    bowlers.update({player_name:rowx})
    
for rowx in range(2,stadium.max_row+1):
    stadium_name = str(stadium.cell(row=rowx,column=1).value)
    #stadium_name = stadium_name[0:1] +" "+ stadium_name[len(stadium_name)-5 : len(stadium_name)+1]
    stadiums.update({stadium_name:rowx})

In [106]:
bowlers_col,stadiums_col=dict(),dict()
for colx in range(2,bowler.max_column+1):
    key = str(bowler.cell(row=1,column=colx).value)
    bowlers_col.update({key:colx})

for colx in range(2,stadium.max_column+1):
    key = str(stadium.cell(row=1,column=colx).value)
    stadiums_col.update({key:colx})

In [107]:
def check_bowler(player_name):
    player_name1 = str(player_name)
    player_name2 = player_name1[0:1] +" "+ player_name1[len(player_name1)-5: len(player_name1)+1]
    
    #try:
    value = bowlers.get(player_name2)
    if value is not None:
        return value
    else:
        data_row=bowler.max_row+1
        bowler.cell(row=data_row, column=1,value= player_name)
        bowlers.update({player_name2 :data_row})
        row = bowlers_col.get(player_name2)
        return row
    
def update_bowler(b,col,val):
    rowx= check_bowler(b)    
    if rowx is not None:
        raw_data= bowler.cell(row=rowx, column=bowlers_col.get(col)).value
        if raw_data is not None:
            data = int(raw_data)
        else:
            data=0
        data+=int(val)
        bowler.cell(row=rowx, column=bowlers_col.get(col),value=data)
        
def check_stadium(name):
    value = stadiums.get(name)
    if value is not None:
        return value
    else:
        data_row=stadium.max_row+1
        stadium.cell(row=data_row, column=1,value= name)
        stadiums.update({name :data_row})
        row = stadiums_col.get(name)
        return row
    
def update_stadium(s,col,val):
    rowx= check_stadium(s)    
    if rowx is not None:
        raw_data= stadium.cell(row=rowx, column=stadiums_col.get(col)).value
        if raw_data is not None:
            data = int(raw_data)
        else:
            data=0
        data+=int(val)
        stadium.cell(row=rowx, column=stadiums_col.get(col),value=data)

In [ ]:
input_file=pd.read_csv("inputFile.csv")


In [108]:

#batting team scored/bowling team conceded in pp
bw,s="",""
w,runs,i=0,0,2

for rowx in range(41595,matches.max_row+1):
    year= int(str(matches.cell(row=rowx,column=2).value)[0:4])
    if  2011<= int(year)<= 2021:
        ball = float(matches.cell(row=rowx,column=6).value)
        innings=matches.cell(row=rowx,column=5).value
        k=int(ball)
        if k<6 and innings<3:
            
            #innings change data
            if innings!=i:                
                #update stadium score,stadium wickets and stadium innings to +1
                update_stadium(s,'runs'+str(i),runs)
                update_stadium(s,'wickets'+str(i),w)
                update_stadium(s,'innings'+str(i),1)
                
                #reset values
                w,runs = 0,0
                i = innings
                s=matches.cell(row=rowx,column=4).value
                   
            #runs
            runs_off_bat=matches.cell(row=rowx,column=12).value
            extras=matches.cell(row=rowx,column=13).value
            update_bowler(bw,'runs',runs_off_bat+extras)
            runs=runs+ runs_off_bat+ extras
            
            #wickets
            if matches.cell(row=rowx,column=19).value is not None:
                update_bowler(bw,'wickets',1)
                w+=1

In [109]:
wb_stadium.save("stadium_data.xlsx")